In [1]:
%run ../widgets/config_check.py

In [2]:
# <api>
%matplotlib inline
import os
path = os.getcwd()
s = '/'
pardir = s.join(path.split(s)[:-1])

import numpy as np

# Load widgets
from jupyter_cms.loader import load_notebook
GD_widgets = load_notebook(str(pardir + '/widgets/2D_gradientdescent_widget.ipynb'))
GD_methods = load_notebook(str(pardir + '/widgets/GradientDescent_methods.ipynb'))
Widget_targets = load_notebook(str(pardir + '/widgets/Widget_targets.ipynb'))
#Norm2D = Widget_targets.Norm2D

# Variants of gradient descent

Vanilla gradient descent slows down to a crawl if the gradient becomes very flat. This can for example be observed on the elliptic normal example. As soon as the elongated vallee is reached, progress along it becomes very slow. Increasing the learning rate helps to some extend, but leads to oscillations of increasing amplitude. The problem being that the gradient and curvature in different directions is of different magnitude. While the learning rate has to be set such that steps are not too large in the steeper/more curved direction thereby slowing the speed of convergence along the shallower direction.

Many variants of gradient descent try to remedy this problem by adjusting the gradient steps or learning rate. Here, we will illustrate several well-known methods. Even though we don't prove it here, all these methods are also suitable for stochastic gradient descent.

## Momentum

The *momentum method* is motivated by a physical analogy: Vanilla gradient can be illustrated by a walker stepping down a hill. At each point, the local direction of steepest descent is found and then a small step in this direction is taken. We could also consider a ball rolling down a hill. In this case, a force acts on the ball in the direction of steepest descent. Without friction, the ball is thereby accelerated and speeds up as long it is rolling down.

Momentum consider the update step as $\mathbf{\theta}_{n+1} = \mathbf{\theta}_n - \mathbf{v}_{n+1}$ with instantaneous velocity $\mathbf{v}_{n+1}$. Following the physical picture the velocity is updated as
$$ \mathbf{v}_{n+1} = \gamma \mathbf{v}_n + \lambda \nabla \mathcal{L}(\mathbf{\theta}_n) . $$

In [3]:
#<api>
class MomentumGD(GD_methods.GradientDescent):
    '''Gradient Descent with momentum.'''

    def __init__(self, target=Widget_targets.MultNorm(), gamma=0.9, learning_rate=0.1,
                num_epochs=20, theta_start=None, stochastic=0):
        
        self.gamma = gamma
        self.velocity = np.zeros_like(theta_start, dtype='float')

        super(MomentumGD, self).__init__(target, learning_rate, num_epochs, theta_start, stochastic)
        
    def __str__(self):
        return "\nMomentum gradient descent has performed "+\
            "%d steps with %d lieing within the depicted boundaries." % (self.num_samples, self.accepted)       
            
    def comp_update(self, theta):
        
        grad = self.target_GD.grad(theta)
                
        update = self.gamma * self.velocity + self.learning_rate * grad
        
        self.velocity = update
        
        return update

In [4]:
widget = GD_widgets.GradientDescentWidget(target='MN2', method='MomentumGDM')
widget.show()

Momentum leads to faster progress along shallow directions as the updates get larger if successive gradients point in the same direction. At the same time, oscillations are dampened as updates gets smaller if the gradient changes sign frequently.

### Questions and suggestions for exploration

1. What happens if you set $\gamma = 0$?
2. Explore the interplay between $\gamma$ and the learning rate $\lambda$.

## Nesterov accelerated gradient descent

The momentum method chooses the new speed based on past gradients, i.e. at the current position. Ideally, we would like our ball to slow down already before running up a hill. *Nesterov accelerated gradient descent* adjusts the update step to take into account the gradient at the new position, that is after the update step, to achieve this effect.

In formulas, its update step is given by
$$ \begin{align*} \mathbf{v}_{n+1} &= \gamma \mathbf{v}_n + \lambda \nabla \mathcal{L}(\mathbf{\theta}_n - \gamma \mathbf{v}_n) \\
   \mathbf{\theta}_{n+1} &= \mathbf{\theta}_n - \mathbf{v}_{n+1} \end{align*}$$
where $\mathbf{\theta}_n - \gamma \mathbf{v}_n$ estimates the new position without taking into account the most recent gradient information.

In [5]:
#<api>
class NesterovGD(GD_methods.GradientDescent):
    '''Nesterov accelerated gradient descent'''
    
    def __init__(self, target=Widget_targets.MultNorm(), gamma=0.9, learning_rate=0.1,
                num_epochs=20, theta_start=None, stochastic=0):
        
        self.gamma = gamma
        self.velocity = np.zeros_like(theta_start, dtype='float')
        
        super(NesterovGD, self).__init__(target, learning_rate, num_epochs, theta_start, stochastic)
    
    def __str__(self):
        return "\nNesterov gradient descent has performed "+\
            "%d steps with %d lieing within the depicted boundaries." % (self.num_samples, self.accepted)
            
    def comp_update(self, theta):
        
        update = self.gamma * self.velocity + \
                    self.learning_rate * self.target_GD.grad(theta - self.gamma * self.velocity)
        
        self.velocity = update
            
        return update

In [6]:
widget = GD_widgets.GradientDescentWidget(target='MN2', method='NesterovGDM')
widget.show()

## Adagrad

The Adagrad method introduces another class of adaptive algorithm which adapt the learning rate for each parameter individually. This provides an alternative way to account for different steepness/curvature in different directions.

Adagrad uses the following per parameter update:
$$ \theta_{i,n+1} = \theta_{i,n} - \frac{\lambda}{\sqrt{g_{i,n}^2 + \epsilon}} $$
Here, $g_{i,n}^2 = \sum_{k = 1}^n \left( \frac{\partial}{\partial \theta_{i,k}} \mathcal{L}(\mathbf{\theta}_k) \right)^2$ is the sum of the square of all gradients wrt $\theta_i$ of all previous time steps and $\epsilon$ prevents division by zero. Thus, the base learning rate is adjusted to the overall size of the gradient of each parameter.

Adagrad has been found to be particularly efficient in problems with sparse data. In this case, some parameters are updated frequently -- with small learning rate -- while others are updated only very infrequently -- but with correspondingly larger learning rate.

In [7]:
#<api>
class ADAGRAD(GD_methods.GradientDescent):
    '''Adagrad'''
    
    def __init__(self, target=Widget_targets.MultNorm(), epsilon=1E-8, learning_rate=0.01,
                num_epochs=20, theta_start=None, stochastic=0):

        self.epsilon = epsilon
        self.past_sq_grad = np.zeros_like(theta_start, dtype='float')
        
        super(ADAGRAD, self).__init__(target, learning_rate, num_epochs, theta_start, stochastic)
    
    def __str__(self):
        return "\ADAGRAD has performed "+\
            "%d steps with %d lieing within the depicted boundaries." % (self.num_samples, self.accepted)
            
    def comp_update(self, theta):
        
        grad = self.target_GD.grad(theta)
        self.past_sq_grad += np.power(grad, 2)
        
        update = (self.learning_rate/(np.sqrt(self.past_sq_grad) + self.epsilon)) * grad
            
        return update

In [8]:
widget = GD_widgets.GradientDescentWidget(target='MN2', method='ADAGRADM')
widget.show()

### Questions and suggestions for exploration

1. Explore the influence of the initial learning rate. What differences to vanilla gradient descent do you observe?
2. Run Adagrad for a larger number of epochs. What do you observe? Reconsider the update rule and explain your observation.

## RMSProp

RMSprop is very similar to Adagrad, but instead of summing the squares of all past gradients, it uses a running average, i.e.
$$ g_{i,n}^2 = \gamma g_{i,n-1}^2 + (1 - \gamma) \left( \frac{\partial}{\partial \theta_i} \mathcal{L}(\mathbf{\theta}) \right)^2 . $$

This way, the adaptation to the learning rate depends more strongly on recent gradients with vanish influence of gradients encountered far in the past. Furthermore, the effective learning rate $\frac{\lambda}{\sqrt{g_{i,n}^2 + \epsilon}}$ can increase again if a shallow region with several small gradients is hit. 

In [9]:
#<api>
class RMSProp(GD_methods.GradientDescent):
    '''Root Mean Square propagation'''

    def __init__(self, target=Widget_targets.MultNorm(), gamma=0.9, epsilon=1E-8, learning_rate=0.1,
                num_epochs=20, theta_start=None, stochastic=0):  

        self.gamma = gamma
        self.epsilon = epsilon
        self.avg_sq_grad = np.zeros_like(theta_start, dtype='float')
        
        super(RMSProp, self).__init__(target, learning_rate, num_epochs, theta_start, stochastic)
        
    def __str__(self):
        return "\nRMSProp has performed "+\
            "%d steps with %d lieing within the depicted boundaries." % (self.num_samples, self.accepted)       
    
    def comp_update(self, theta):
        
        grad = self.target_GD.grad(theta)                    
        self.avg_sq_grad = self.gamma * self.avg_sq_grad + (1-self.gamma) * np.power(grad, 2)
        
        update = (self.learning_rate/(np.sqrt(self.avg_sq_grad) + self.epsilon)) * grad
            
        return update

In [10]:
widget = GD_widgets.GradientDescentWidget(target='MN2', method='RMSPropM')
widget.show()

### Questions and suggestions for exploration

1. Explore the differences between RMSprop and Adagrad in more detail. Can you observe the effect that the effective learning rate increases during the course of RMSprop?
2. Investigate the influence of $\gamma$. What happens for $\gamma \to 0$ and $\gamma \to 1$?

## Adam

Adam keeps track of both, past gradients, as well as their magnitudes, squares of past gradients. To this end, two internal variables are updated to compute running averages, i.e.
$$ \begin{align*}
     m_{1, n+1} &= \beta_1 m_{1, n} + (1 - \beta_1) \nabla \mathcal{L}(\mathbf{\theta}) \\
     m_{2, n+1} &= \beta_2 m_{2, n} + (1 - \beta_2) \left( \nabla \mathcal{L}(\mathbf{\theta}) \right)^2 \\
   \end{align*}
$$
where the square of the gradient is understood pointwise, i.e. $\left( \nabla \mathcal{L}(\mathbf{\theta}) \right)^2_i = \left( \frac{\partial}{\partial \theta_i} \mathcal{L}(\mathbf{\theta}) \right)^2$.

As $m_{1,0} = m_{2,0} = 0$ are initialized at zero, initial gradient estimates are biased towards zero. Adam corrects this bias before applying the estimates in the final learning rate:
$$ \theta_{i,n+1} = \theta_{i,n} - \frac{\lambda}{\sqrt{\frac{m_{2,n+1}}{1 - \beta_2^{n+1}} + \epsilon}} \frac{m_{1,n+1}}{1 - \beta_1^{n+1}} . $$

Note that the bias correction gets smaller and smaller for later time steps, i.e. $1 - \beta_{1,2}^{n+1} \rightarrow_{n \to \infty} 1$, as it should be. In this regard, Adam can be considered as a combination of RMSprop and momentum.

In [11]:
#<api>
class ADAM(GD_methods.GradientDescent):
    '''Adaptive Moment Estimation'''

    def __init__(self, target=Widget_targets.MultNorm(), beta1=0.9, beta2=0.999, epsilon=1E-8,
                 learning_rate=0.1, num_epochs=20, theta_start=None, stochastic=0):
        
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        
        # initialize moment estimates
        self.est_mom_1 = np.zeros_like(theta_start, dtype='float')
        self.est_mom_2 = np.zeros_like(theta_start, dtype='float')


        super(ADAM, self).__init__(target, learning_rate, num_epochs, theta_start, stochastic)

        
    def __str__(self):
        return "\nADAM has performed "+\
            "%d steps with %d lieing within the depicted boundaries." % (self.num_samples, self.accepted)       
        
    def comp_update(self, theta):
        
        grad = self.target_GD.grad(theta)
                    
        self.est_mom_1 = self.beta1 * self.est_mom_1 + (1-self.beta1) * grad
        self.est_mom_2 = self.beta2 * self.est_mom_2 + (1-self.beta2) * np.power(grad, 2)
            
        # bias corrected decaying averages
        unbiased_est_mom_1 = self.est_mom_1/(1 - np.power(self.beta1, self.epochID+1))
        unbiased_est_mom_2 = self.est_mom_2/(1 - np.power(self.beta2, self.epochID+1))

        update = self.learning_rate/(np.sqrt(unbiased_est_mom_2) + self.epsilon) \
                                * unbiased_est_mom_1
            
        return update

In [12]:
widget = GD_widgets.GradientDescentWidget(target='MN2', method='ADAMM')
widget.show()

Currently, Adam and RMSprop appear to provide a good compromise between robustness -- wrt to the choice of parameters -- and speed of convergence. Both methods are often used in practical applications. Especially, in high-dimensional problems large differences between algorithms can be observed. In this case, many models exhibit loss functions with saddle points. At these points gradients vanish even though they are not local minima, i.e. there are directions along hich the loss increases, as well as directions along which it decreases. A good algorithm should be quickly able to identify the decreasing directions without slowing down due to shallow gradients close to the saddle. 

### Questions and suggestions for exploration
1. Investigate the influence of the parameters $\beta_1, \beta_2$ in the Adam method. Explore its behavior for a range of different targets.
1. Compare different algorithms around a saddle point. Is there any algorithm which can escape from the saddle point when the inital condition is set at $(\theta_{1,0}, 0)$? 